In [22]:
import pandas as pd

In [23]:
df = pd.read_excel("DATA/People_list.xlsx")
#df = df[df["Category"] == "Activists"]

In [24]:
asked_df = pd.DataFrame(columns= df.columns)

In [25]:
category_counts = df['Category'].value_counts()
total_counts = category_counts.sum()
unique_categories = category_counts.index.tolist()
categories_share = (category_counts / total_counts).tolist()

In [26]:
difficulty = 1

In [27]:
def probabilistic_round_selection(possible_questions, asked_df, unique_categories, categories_share, difficulty):
    round_questions = pd.DataFrame()
    selected_questions = pd.DataFrame()

    patterns =  [[0.6,0.3,0.1],[0.4,0.4,0.2],[0.2,0.5,0.3]]

    difficulty_levels = ['Level 1', 'Level 2', 'Level 3']  # Adjust as necessary
    difficulty_share = patterns[difficulty - 1]  # This should be a list containing share of each level e.g [0.6, 0.3, 0.1] for ['Level 1', 'Level 2', 'Level 3']
    num_questions = len(possible_questions)

    for i, category in enumerate(unique_categories):
        for j, level in enumerate(difficulty_levels):
            category_level_share = categories_share[i] * difficulty_share[j]

            category_questions = possible_questions[(possible_questions['Category'] == category) & (possible_questions['Fame_Level'] == level)]
            asked_questions = asked_df[(asked_df['Category'] == category) & (asked_df['Fame_Level'] == level)]

            if len(category_questions) > 0:
                question_df = category_questions.sample(frac = category_level_share, replace = False)
                round_questions = pd.concat([round_questions, question_df])

                asked_df = pd.concat([asked_df, question_df])
                selected_questions = pd.concat([selected_questions, question_df])

            elif len(asked_questions) > 0:
                rollback_questions = asked_df[(asked_df['Category'] == category) & (asked_df['Fame_Level'] == level)]
                question_df = rollback_questions.sample(frac = category_level_share, replace = False)

                round_questions = pd.concat([round_questions, question_df])

                # Drop rolled back questions from asked_df
                asked_df.drop(rollback_questions.index, inplace = True)
                selected_questions = pd.concat([selected_questions, question_df])

    return round_questions, asked_df, selected_questions

In [28]:
a, b, c = probabilistic_round_selection(df, asked_df, unique_categories, categories_share, difficulty)

In [30]:
possible_questions = df


round_questions = pd.DataFrame()
selected_questions = pd.DataFrame()

patterns =  [[0.6,0.3,0.1],[0.4,0.4,0.2],[0.2,0.5,0.3]]

difficulty_levels = ['Level 1', 'Level 2', 'Level 3']  # Adjust as necessary
difficulty_share = patterns[difficulty - 1]  # This should be a list containing share of each level e.g [0.6, 0.3, 0.1] for ['Level 1', 'Level 2', 'Level 3']
num_questions = len(possible_questions)

for i, category in enumerate(unique_categories):
    for j, level in enumerate(difficulty_levels):
        category_level_share = categories_share[i] * difficulty_share[j]

        category_questions = possible_questions[(possible_questions['Category'] == category) & (possible_questions['Fame_Level'] == level)]
        asked_questions = asked_df[(asked_df['Category'] == category) & (asked_df['Fame_Level'] == level)]

        if len(category_questions) > 0:
            question_df = category_questions.sample(frac = category_level_share, replace = False)
            round_questions = pd.concat([round_questions, question_df])

            asked_df = pd.concat([asked_df, question_df])
            selected_questions = pd.concat([selected_questions, question_df])

        elif len(asked_questions) > 0:
            rollback_questions = asked_df[(asked_df['Category'] == category) & (asked_df['Fame_Level'] == level)]
            question_df = rollback_questions.sample(frac = category_level_share, replace = False)

            round_questions = pd.concat([round_questions, question_df])

            # Drop rolled back questions from asked_df
            asked_df.drop(rollback_questions.index, inplace = True)
            selected_questions = pd.concat([selected_questions, question_df])


